In [5]:
from bs4 import BeautifulSoup as bs
from bs4 import Tag, NavigableString
import re
import requests as r
import pandas as pd
import numpy as np
import datetime
import csv
import os
import sys
from selenium import webdriver
import censusdata as cd
import geopy
import seaborn as sns

### Scrape Traffic Stop Data

In [36]:
myurl = 'https://openpolicing.stanford.edu/data/'
html = r.get(myurl)
soup = bs(html.content,'lxml')
current_link = ''
for l in soup.find_all('a'):
    current_link = l.get('href')
    if 'tx_austin' in current_link and 'csv' in current_link:
        print(current_link)
        xlsx_file = r.get(current_link)
        output = open('traffic_stops_aus.csv', 'wb')
        output.write(xlsx_file.content)
        output.close()

https://stacks.stanford.edu/file/druid:yg821jf8611/yg821jf8611_tx_austin_2020_04_01.csv.zip


In [38]:
aus = pd.read_csv('traffic_stops_aus.csv')

In [39]:
aus.head()

,raw_row_number,date,subject_age,subject_race,subject_sex,officer_id_hash,type,contraband_found,contraband_drugs,contraband_weapons,...,vehicle_year,raw_ethnicity,raw_person_search_search_based_on,raw_person_search_search_discovered,raw_person_searched,raw_vehicle_search_search_based_on,raw_vehicle_search_search_discovered,raw_vehicle_searched,raw_race_description,raw_street_check_description
0,141,2006-01-07,53.0,white,female,59c9092a79,vehicular,False,False,False,...,1996.0,NaN,NaN,NaN,YES = -,NaN,NaN,YES = -,WHITE,WARNING
1,4819,2006-03-21,21.0,white,male,46f8ed46eb,vehicular,NaN,NaN,NaN,...,2000.0,NaN,NaN,NaN,NO = -,NaN,NaN,NO= -,WHITE,WARNING
2,4820,2006-03-21,43.0,black,female,0d1d4fb150,vehicular,NaN,NaN,NaN,...,2003.0,NaN,NaN,NaN,NO = -,NaN,NaN,NO= -,BLACK,WARNING
3,4821,2006-04-20,46.0,hispanic,male,c70bca9788,NaN,NaN,NaN,NaN,...,1994.0,H,NaN,NaN,NO = -,NaN,NaN,NO= -,WHITE,WARNING
4,4822,2006-03-21,25.0,white,male,e3f693e058,vehicular,NaN,NaN,NaN,...,2002.0,NaN,NaN,NaN,NO = -,NaN,NaN,NO= -,WHITE,WARNING


In [42]:
races = list(aus['subject_race'].unique())
sexes = list(aus['subject_sex'].unique())

In [49]:
aus.columns

Index(['raw_row_number', 'date', 'subject_age', 'subject_race', 'subject_sex',
       'officer_id_hash', 'type', 'contraband_found', 'contraband_drugs',
       'contraband_weapons', 'frisk_performed', 'search_conducted',
       'search_person', 'search_vehicle', 'search_basis', 'reason_for_stop',
       'vehicle_make', 'vehicle_model', 'vehicle_registration_state',
       'vehicle_year', 'raw_ethnicity', 'raw_person_search_search_based_on',
       'raw_person_search_search_discovered', 'raw_person_searched',
       'raw_vehicle_search_search_based_on',
       'raw_vehicle_search_search_discovered', 'raw_vehicle_searched',
       'raw_race_description', 'raw_street_check_description'],
      dtype='object')

In [58]:
data = aus.groupby(['subject_sex','subject_race']).agg(['count']).reset_index()

In [59]:
data

,subject_sex,subject_race,raw_row_number,date,subject_age,officer_id_hash,type,contraband_found,contraband_drugs,contraband_weapons,...,vehicle_year,raw_ethnicity,raw_person_search_search_based_on,raw_person_search_search_discovered,raw_person_searched,raw_vehicle_search_search_based_on,raw_vehicle_search_search_discovered,raw_vehicle_searched,raw_race_description,raw_street_check_description
,,,count,count,count,count,count,count,count,count,...,count,count,count,count,count,count,count,count,count,count
0,female,asian/pacific islander,4552,4552,4515,4552,4359,28,28,28,...,3683,2783,26,70,4062,11,50,3698,4552,4552
1,female,black,19482,19482,19334,19482,17761,755,755,755,...,15959,12478,553,669,16856,534,622,15161,19482,19482
2,female,hispanic,27285,27285,27049,27285,24441,943,943,943,...,23484,27285,721,837,23059,648,729,19946,27284,27285
3,female,other,466,466,461,466,443,7,7,7,...,433,412,5,9,423,2,4,356,466,466
4,female,unknown,791,791,763,791,747,15,15,15,...,663,453,9,18,694,10,14,626,791,791
5,female,white,90287,90287,89687,90287,84980,1229,1229,1229,...,71408,48662,829,1566,80688,674,1329,75405,90287,90286
6,male,asian/pacific islander,8613,8613,8558,8613,7894,163,163,163,...,6796,5079,161,243,7640,63,132,6978,8613,8613
7,male,black,52836,52836,52558,52836,37005,4316,4316,4316,...,40431,32493,4172,4397,47161,2277,2450,43254,52836,52836
8,male,hispanic,96646,96646,96087,96646,56258,6114,6114,6114,...,75445,96646,5672,5891,86893,2883,3063,80114,96646,96646


### Census Data

In [60]:
df = pd.DataFrame(cd.search('acs1', 2018, 'concept', 'sex by age'))

In [61]:
df.head()

,0,1,2
0,B01001A_001E,SEX BY AGE (WHITE ALONE),Estimate!!Total
1,B01001A_002E,SEX BY AGE (WHITE ALONE),Estimate!!Total!!Male
2,B01001A_003E,SEX BY AGE (WHITE ALONE),Estimate!!Total!!Male!!Under 5 years
3,B01001A_004E,SEX BY AGE (WHITE ALONE),Estimate!!Total!!Male!!5 to 9 years
4,B01001A_005E,SEX BY AGE (WHITE ALONE),Estimate!!Total!!Male!!10 to 14 years
